In [1]:
import os
import pandas as pd
import numpy as np
import torch
import plotly.express as px
import plotly.graph_objs as go
import plotly.colors
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from umap import UMAP

from src.ModelUtils import (load_model,
                            train_model,
                            save_model_losses,
                            load_and_plot_losses)
from src.DataUtils import (load_data_from_directory,
                           segment_all_steps,
                           steps_to_tensor)

In [ ]:
## Directories and constants
FIGURES_DIR = './figures/SCI'
MODELS_DIR = './src/models'
DATA_DIR = './csv/SCI_Plot_Features'
HEALTHY_KEY = 'Pre'
SICK_KEY = 'Post'
SIDE_KEY = ('Left','Right')

## Hyperparameters and early stopping
INPUT_DIM = None
HIDDEN_DIM = 128#64
LATENT_DIM = 64#16
BATCH_SIZE = 32
NUM_EPOCHS = 5000
LR = 1e-3
PATIENCE = 100 # number of epochs to wait for improvement before stopping
MIN_DELTA = 1e-4 # minimum change to qualify as an improvement
BEST_MODEL_PATH = None#os.path.join(MODELS_DIR, 'lstm_VAE_no_first_last_20250609_121841.pt')
BEST_LOSS_PATH = os.path.join(FIGURES_DIR, 'SCI_losses.csv')

## Plot constants
SCATTER_SIZE = 6
SCATTER_LINE_WIDTH = 1
SCATTER_SYMBOL = 'circle'
LEGEND_FONT_SIZE = 18
TITLE_FONT_SIZE = 24
AXIS_FONT_SIZE = 16
AXIS_TITLE_FONT_SIZE = 20

# Load the data
data = load_data_from_directory(DATA_DIR)

# Print the data structure
for datagroup, mice in data.items():
    print(f"Group: {datagroup}")
    for mouse_direction, runs in mice.items():
        print(f"\t{mouse_direction}: {len(runs)} runs with shapes: {[df.shape for df in runs.values()]}")

Group: SCI_PostSCI
	Left_Mouse100: 4 runs with shapes: [(141, 199), (161, 199), (241, 199), (201, 199)]
	Left_Mouse102: 1 runs with shapes: [(381, 199)]
	Left_Mouse104: 3 runs with shapes: [(211, 199), (201, 199), (315, 199)]
	Left_Mouse110: 3 runs with shapes: [(341, 199), (251, 199), (237, 199)]
	Left_Mouse118: 3 runs with shapes: [(299, 199), (175, 199), (196, 199)]
	Left_Mouse122: 1 runs with shapes: [(421, 199)]
	Left_Mouse132: 3 runs with shapes: [(401, 199), (331, 199), (231, 199)]
	Left_Mouse134: 3 runs with shapes: [(271, 199), (261, 199), (181, 199)]
	Left_Mouse138: 3 runs with shapes: [(491, 199), (374, 199), (471, 199)]
	Left_Mouse142: 3 runs with shapes: [(261, 199), (163, 199), (171, 199)]
	Left_Mouse148: 1 runs with shapes: [(901, 199)]
	Left_Mouse14: 6 runs with shapes: [(244, 199), (161, 199), (201, 199), (201, 199), (105, 199), (106, 199)]
	Left_Mouse150: 3 runs with shapes: [(401, 199), (541, 199), (531, 199)]
	Left_Mouse154: 3 runs with shapes: [(281, 199), (161, 19

In [3]:
# Print the features count
for datagroup, mice in data.items():
    for mouse_direction, runs in mice.items():
        for run, df in runs.items():
            print(f"Number of features: {df.shape[1] + 1 - 4}") # +1 bc of index, -4 to exclude frame, forestep, hindstep, and time
            break
        break
    break

Number of features: 196


In [4]:
## Segment all steps in the data
# This will create two dictionaries: segmented_hindsteps and segmented_foresteps
# Each dictionary will contain segmented steps for each mouse direction and run
    # "step": step_df,
    # "group": group,
    # "mouse": mouse,
    # "run": run
segmented_hindsteps, segmented_foresteps = segment_all_steps(data)

## Flatten all steps into a single array to compute global mean/std and get the scaler
all_healthy_arrays = [step_dict["step"].values for step_dict in segmented_hindsteps if HEALTHY_KEY in step_dict["group"] and SIDE_KEY[0] in step_dict["mouse"]]
flat_data = np.vstack(all_healthy_arrays)

scaler = StandardScaler()
scaler = scaler.fit(flat_data)

## Prepare the data for training
healthy_steps = [s for s in segmented_hindsteps if HEALTHY_KEY in s["group"] and SIDE_KEY[0] in s["mouse"]]
step_tensor, lengths = steps_to_tensor(healthy_steps, scaler)
if INPUT_DIM is None:
    INPUT_DIM = step_tensor.shape[2]

print(f"Step tensor shape: {step_tensor.shape}, \nLengths shape: {lengths.shape}")

Step tensor shape: torch.Size([1244, 290, 196]), 
Lengths shape: torch.Size([1244])


In [5]:
### Training block
model = None
if BEST_MODEL_PATH:
    model = load_model(model_path=BEST_MODEL_PATH, input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM)
    load_and_plot_losses(losses_path=BEST_LOSS_PATH,
                        title_font_size=TITLE_FONT_SIZE,
                        axis_title_font_size=AXIS_TITLE_FONT_SIZE,
                        legend_font_size=LEGEND_FONT_SIZE)
else:
    model, train_losses, val_losses = train_model(
        step_tensor=step_tensor,
        lengths=lengths,
        input_dim=INPUT_DIM,
        hidden_dim=HIDDEN_DIM,
        latent_dim=LATENT_DIM,
        batch_size=BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        lr=LR,
        patience=PATIENCE,
        min_delta=MIN_DELTA,
        models_dir=MODELS_DIR
    )
    save_model_losses(train_losses, val_losses, figures_dir=FIGURES_DIR,
                        title_font_size=TITLE_FONT_SIZE,
                        axis_title_font_size=AXIS_FONT_SIZE,
                        legend_font_size=LEGEND_FONT_SIZE)

assert model is not None, "Model training failed or no model was loaded."

Using device: cuda
VAE_t device: cuda


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1, Train: 8.9695 - Val: 8.9254 - Time: 0.73s
Epoch 2, Train: 8.9134 - Val: 8.8371 - Time: 0.62s
Epoch 3, Train: 8.8028 - Val: 8.7440 - Time: 0.62s
Epoch 4, Train: 8.7126 - Val: 8.6637 - Time: 0.61s
Epoch 5, Train: 8.6549 - Val: 8.6002 - Time: 0.62s
Epoch 6, Train: 8.5874 - Val: 8.5472 - Time: 0.62s
Epoch 7, Train: 8.5360 - Val: 8.4949 - Time: 0.62s
Epoch 8, Train: 8.4841 - Val: 8.4521 - Time: 0.60s
Epoch 9, Train: 8.4523 - Val: 8.4151 - Time: 0.60s
Epoch 10, Train: 8.4241 - Val: 8.3864 - Time: 0.59s
Epoch 11, Train: 8.3949 - Val: 8.3620 - Time: 0.59s
Epoch 12, Train: 8.3720 - Val: 8.3398 - Time: 0.59s
Epoch 13, Train: 8.3540 - Val: 8.3284 - Time: 0.61s
Epoch 14, Train: 8.3391 - Val: 8.3109 - Time: 0.60s
Epoch 15, Train: 8.3227 - Val: 8.2970 - Time: 0.60s
Epoch 16, Train: 8.3091 - Val: 8.2855 - Time: 0.61s
Epoch 17, Train: 8.3045 - Val: 8.2783 - Time: 0.60s
Epoch 18, Train: 8.2932 - Val: 8.2699 - Time: 0.59s
Epoch 19, Train: 8.2884 - Val: 8.2625 - Time: 0.59s
Epoch 21, Train: 8.27

In [6]:
### Evaluate the model on pathological steps
selected_steps = [
    s for s in segmented_hindsteps 
    if SIDE_KEY[0] in s["mouse"] 
]

step_tensor_all, lengths_all = steps_to_tensor(selected_steps, scaler)

## Get all embeddings for selected steps
model.eval()
with torch.no_grad():
    _, mu_t, _ = model(step_tensor_all, lengths_all)  # (B, T, D)

# Convert each step’s latent sequence to list of (T_i, D)
mu_t_masked = [
    mu_t[i, :lengths_all[i]].cpu().numpy()  # shape: (T_i, D)
    for i in range(mu_t.shape[0])
]

# Stack all latent vectors across all steps
all_latents = np.concatenate(mu_t_masked, axis=0)  # shape: (total_timepoints, D)

# Fit UMAP
umap_coords = UMAP(n_components=3, random_state=42).fit_transform(all_latents)

# Now split the coordinates back by original steps
umap_split = []
idx = 0
for i in range(len(mu_t_masked)):
    length = mu_t_masked[i].shape[0]
    umap_split.append(umap_coords[idx:idx + length])  # shape: (T_i, 3)
    idx += length

c:\Users\carme\Documents\GitHub\MScThesis\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



OSError: exception: access violation reading 0x00000001BDA838E1

In [ ]:
datasets = sorted(set(s["group"] for s in selected_steps))
color_map = {ds: color for ds, color in zip(datasets, plotly.colors.qualitative.Plotly)}
# if pre is in group, use the first color, otherwise use the second color
for i, ds in enumerate(datasets):
    if "pre" in ds:
        color_map[ds] = plotly.colors.qualitative.Plotly[0]
    else:
        color_map[ds] = plotly.colors.qualitative.Plotly[1]

fig = go.Figure()
legend_shown = defaultdict(bool)

for i, umap_seq in enumerate(umap_split):
    step_meta = selected_steps[i]
    dataset = step_meta["group"]
    mouse = step_meta["mouse"]
    run = step_meta["run"]
    color = color_map.get(dataset, "gray")
    show_legend = not legend_shown[dataset]
    legend_shown[dataset] = True

    fig.add_trace(go.Scatter3d(
        x=umap_seq[:, 0],
        y=umap_seq[:, 1],
        z=umap_seq[:, 2],
        mode='lines+markers',
        name=dataset if show_legend else None,
        legendgroup=dataset,
        showlegend=show_legend,
        line=dict(width=2, color=color),
        marker=dict(size=2, color=color),
        hoverinfo='text',
        text=[f"{dataset} | {mouse} | run={run} | t={t}" for t in range(len(umap_seq))]
    ))

fig.update_layout(
    title="Latent Trajectories Over Time by Dataset",
    scene=dict(
        xaxis_title='UMAP1',
        yaxis_title='UMAP2',
        zaxis_title='UMAP3'
    ),
    legend=dict(title="Dataset"),
    width=900,
    height=700,
    template='plotly_white'
)

fig.show()

In [ ]:
def plot_timewise_umap_trajectories(segmented_hindsteps, scaler):
    """
    Plot UMAP projection of all timepoints in each step.
    Each step becomes a trajectory (line) in 3D UMAP space.
    Mice from the same dataset (e.g., 'pre', 'post') share color and legend.
    """
    from collections import defaultdict

    filtered_steps = [s for s in segmented_hindsteps if "left" in s["mouse"]]

    step_id = 0
    timepoints = []
    metadata = []

    for s in filtered_steps:
        arr = scaler.transform(s["step"].values)  # (T, D)
        for t in range(arr.shape[0]):
            timepoints.append(arr[t])
            metadata.append({
                "step_id": step_id,
                "t": t,
                "mouse": s["mouse"],
                "group": s["group"],
                "run": s["run"]
            })
        step_id += 1

    X = np.stack(timepoints)
    umap_coords = UMAP(n_components=3, random_state=42).fit_transform(X)

    umap_df = pd.DataFrame(umap_coords, columns=["UMAP1", "UMAP2", "UMAP3"])
    umap_df = pd.concat([umap_df, pd.DataFrame(metadata)], axis=1)

    datasets = sorted(umap_df["group"].unique())
    color_map = {ds: color for ds, color in zip(datasets, plotly.colors.qualitative.Plotly)}

    fig = go.Figure()
    legend_shown = defaultdict(bool)

    for step_id, group in umap_df.groupby("step_id"):
        dataset = group["group"].iloc[0]
        color = color_map[dataset]
        show_legend = not legend_shown[dataset]
        legend_shown[dataset] = True

        fig.add_trace(go.Scatter3d(
            x=group["UMAP1"],
            y=group["UMAP2"],
            z=group["UMAP3"],
            mode="lines+markers",
            line=dict(color=color, width=2),
            marker=dict(size=3, color=color),
            name=dataset,
            legendgroup=dataset,
            showlegend=show_legend,
            text=[f"{group['mouse'].iloc[0]} | run={r} | t={t}" for r, t in zip(group["run"], group["t"])]
        ))

    fig.update_layout(
        title="Time-Resolved UMAP of Step Dynamics by Dataset",
        scene=dict(
            xaxis_title="UMAP1",
            yaxis_title="UMAP2",
            zaxis_title="UMAP3"
        ),
        legend=dict(title="Dataset", font=dict(size=12)),
        width=900,
        height=700,
        template="plotly_white"
    )

    fig.show()

# Usage:
plot_timewise_umap_trajectories(segmented_hindsteps, scaler)